In [63]:
import networkx as nx
import pandas as pd

df = pd.read_csv('./data/subset.csv')
df = df.drop(columns=['Unnamed: 0'])
#g = nx.from_pandas_edgelist(df[['Source','Destination']], source='Source', target='Destination',create_using=nx.DiGraph())
g = nx.read_edgelist('./data/sub_graph.csv', create_using=nx.DiGraph(), delimiter=',', data = False)
print(g)

DiGraph with 1000 nodes and 5683 edges


In [64]:
# Page Rank
pr = nx.pagerank(g)
df['Page_Rank_Src'] = df.Source.apply(lambda row: pr.get(str(row)))
df['Page_Rank_Dst'] = df.Destination.apply(lambda row: pr.get(str(row)))

In [65]:
# Shortest Path
def get_shortest_path(x, y):
    x = str(int(x))
    y = str(int(y))
    d = -1
    try:
        if g.has_edge(x, y):
            g.remove_edge(x, y)
            d = nx.shortest_path_length(g, source=x, target=y)
            g.add_edge(x, y)
        else:
            d = nx.shortest_path_length(g, source=x, target=y)
    except:
        d = -1
    return d

df['Shortest_Path'] = df.apply(lambda row: get_shortest_path(row['Source'], row['Destination']), axis = 1)

In [66]:
# Follow Features
followers_src, followers_dst, followees_src, followees_dst, int_followers, int_followees = [], [], [], [], [], []
# TODO
followers_Jaccard_coefficient, followees_Jaccard_coefficient = [], []
followers_Cosine_distance, followees_Cosine_distance = [], []

for i, r in df.iterrows():
    s_node = str(int(r['Source']))
    d_node = str(int(r['Destination']))

    pre_src = set(g.predecessors(s_node)) if set(g.predecessors(s_node)) else set()
    suc_src = set(g.successors(s_node)) if set(g.successors(s_node)) else set()

    pre_dst = set(g.predecessors(d_node)) if set(g.predecessors(d_node)) else set()
    suc_dst = set(g.successors(d_node)) if set(g.successors(d_node)) else set()

    followers_src.append(len(pre_src))
    followees_src.append(len(suc_src))

    followers_dst.append(len(pre_dst))
    followees_dst.append(len(suc_dst))

    int_followers.append(len(pre_src.intersection(pre_dst)))
    int_followees.append(len(suc_src.intersection(suc_dst)))
        
df['Followers_Src'] = followers_src
df['Followees_Src'] = followees_src
df['Followers_Dst'] = followers_dst
df['Followees_Dst'] = followees_dst
df['Int_Followers'] = int_followers
df['Int_Followees'] = int_followees

In [67]:
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,1,6448,1,0.000388,0.000790,2,0,7,1,0,0,0
1,1,1376,1,0.000388,0.000426,-1,0,7,0,3,0,0
2,1,1883,1,0.000388,0.000822,4,0,7,2,30,0,1
3,1,6716,1,0.000388,0.005997,2,0,7,40,30,0,0
4,1,4710,1,0.000388,0.000421,-1,0,7,0,8,0,0


In [68]:
df.to_csv('./data/final_dataset.csv')